In [45]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from numpy import argmax, array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2
from sklearn.metrics import mean_squared_error

In [135]:
MAX_ITRERATION = 10
global input_weights
global bias
global g_best
global p_best
global g_best_x
global p_best_x

In [2]:
train = pd.read_csv("kdd_train.csv")
test = pd.read_csv("kdd_test.csv")

print(test.shape,train.shape)
#print(test['duration'])

(22544, 42) (125973, 42)


# Data Pre-Process


In [3]:
for i in ['protocol_type', 'service', "flag", "labels"]:
    values_train = array(train[i])
    values_test = array(test[i])
    label_encoder = LabelEncoder()
    integer_encoded_train = label_encoder.fit_transform(values_train)
    integer_encoded_test = label_encoder.fit_transform(values_test)
    train[i] = integer_encoded_train
    test[i] = integer_encoded_test

X_train = train.iloc[:, :41]
y_train = train.iloc[:, 41]

X_test = test.iloc[:, :41]
y_test = test.iloc[:, 41]

#print(X_train.head(),y_train.head())
#print(X_test.head(),y_test.head())
#print(list(y_train))#

In [4]:
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=20)
# fit RFE
rfe.fit(X_train, y_train)
# summarize all features
rfe_X_train = pd.DataFrame()
rfe_X_test = pd.DataFrame()
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    col_name_train = train[train.columns[i]]
    col_name_test = test[test.columns[i]]
    if rfe.support_[i]:
        rfe_X_train = pd.concat([rfe_X_train, col_name_train], axis = 1)
        rfe_X_test = pd.concat([rfe_X_test, col_name_test], axis = 1)
      
print (rfe_X_train.shape, rfe_X_test.shape)
#print(col_name_train)

Column: 0, Selected True, Rank: 1.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected False, Rank: 4.000
Column: 7, Selected False, Rank: 6.000
Column: 8, Selected False, Rank: 13.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 2.000
Column: 11, Selected False, Rank: 7.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 12.000
Column: 14, Selected False, Rank: 16.000
Column: 15, Selected False, Rank: 20.000
Column: 16, Selected False, Rank: 10.000
Column: 17, Selected False, Rank: 9.000
Column: 18, Selected False, Rank: 17.000
Column: 19, Selected False, Rank: 21.000
Column: 20, Selected False, Rank: 22.000
Column: 21, Selected False, Rank: 18.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 8.000
Column: 24, Selected True, Rank: 1.000
Column: 25,

# PSO-ELM


In [24]:
input_size = rfe_X_train.shape[1]
hidden_size = 20


weights = np.array([np.random.uniform(-5,5, size=(input_size, hidden_size)) for i in range(25)])
bias = np.array([np.random.uniform(-5,5,hidden_size) for i in range(25)])
velocity_x = np.array([np.array([0 for j in range((input_size*hidden_size)+hidden_size)]) for i in range(25)])

input_weight_x = []

for i in range(25):
    input_weight_x.append(np.concatenate((weights[i].flatten(),bias[i])))

input_weight_x = array(input_weight_x)
print(input_weight_x.shape,bias)

(25, 420) [[ 1.75492243e+00 -1.28850906e+00  8.87428678e-01 -9.31170035e-01
  -1.59653452e+00  4.71486373e+00 -3.89211479e+00 -4.23396287e+00
  -1.02136072e+00  4.47744045e+00  2.85117389e+00  1.28315514e+00
  -2.79715076e+00 -1.64121019e+00  3.04773907e+00  3.32320026e-01
   1.63634480e+00 -3.27284693e+00 -4.43870208e+00  3.12913119e+00]
 [ 3.25809372e+00 -9.53313719e-01  6.92949065e-01 -1.53874135e+00
   7.85431001e-01  1.77306673e+00 -1.71669641e-01  8.31458817e-01
  -1.87914506e+00  3.84107598e+00 -2.99792949e+00 -2.70140262e+00
   2.79978840e+00 -6.44426680e-01 -2.45280151e+00 -4.18340849e+00
  -2.73705355e+00 -3.56921863e-01  2.68124103e+00  3.79436393e+00]
 [-1.16650962e+00  4.49879488e+00 -3.19938887e+00 -7.63869648e-01
   2.82142183e+00  6.56552925e-01 -7.53996308e-01 -6.64573020e-01
  -3.92120329e+00 -2.66002639e+00  3.76872300e+00  1.40599317e+00
  -2.31519270e-01 -9.25657292e-01 -3.06650116e+00  1.14739289e+00
  -5.83972545e-01  2.64498009e+00 -4.14713880e+00  2.47743118e+0

## ELM (Fitness Function)

In [127]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def hidden_nodes(X,input_weights,bias):
    G = np.dot(X, input_weights)
    G = G + bias
    H = sigmoid(G)
    return H

def predict(X,input_weights,output_weights,bias):
    out = hidden_nodes(X,input_weights,bias)
    out = np.dot(out, output_weights)
    return out

def ELM_fitness(row):
    input_weights = row[:400].reshape(input_size,hidden_size)
    bias = row[-hidden_size:]
    output_weights = np.dot(pinv2(hidden_nodes(rfe_X_train,input_weights,bias)), y_train)
    #pred
    prediction = predict(rfe_X_test,input_weights,output_weights,bias)

    return mean_squared_error(y_test,prediction)

In [136]:
p_best, p_best_x = [999 for row in input_weight_x], input_weight_x
g_best, g_best_x = 999,[]
print(p_best,p_best_x.shape)

[999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999] (25, 420)


In [137]:
random_numbers = np.array([np.random.uniform(0,1,2) for i in range(MAX_ITRERATION)])
print(random_numbers)

[[0.46017506 0.18847533]
 [0.2504519  0.48437404]
 [0.48323257 0.30132861]
 [0.5926036  0.73811753]
 [0.02321577 0.59402104]
 [0.9534976  0.41140976]
 [0.99823749 0.17496024]
 [0.1992792  0.93988773]
 [0.22265466 0.66567015]
 [0.01739318 0.76797124]]


## Velocity Calculation

In [14]:
c1 = 1
c2 = 2
w = 0.6
def calculate_v(old_v,c1,c2,r1,r2,x,p_best_x,g_best_x):
  return w*old_v + c1*r1*(p_best_x-x) + c2*r2*(g_best_x-x)

In [138]:
for iteration in range(0,10):
    input_weight_x = input_weight_x + velocity_x
    current_x = [ELM_fitness(row) for row in input_weight_x]
    for i,f_val in enumerate(current_x):
        if p_best[i] > f_val:
            p_best[i] = f_val
            p_best_x[i] = input_weight_x[i]
        if p_best[i] < g_best:
            g_best = p_best[i]
            g_best_x = p_best_x[i]
        v = []
    for i, particle in enumerate(input_weight_x):
        v.append(calculate_v(velocity_x[i],c1,c2,random_numbers[iteration][0], random_numbers[iteration][1],particle,p_best_x[i],g_best_x))
    velocity_x = np.array(v)

ountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))
<ipython-input-127-079125935750>:2: Ru

In [139]:
#print(g_best_x.shape,g_best_x)
print(g_best)

43.43577708372576


# Best Input-Weight And Bias

In [117]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + bias
    H = sigmoid(G)
    return H

input_weights = g_best_x[:400].reshape(input_size,hidden_size)
bias = g_best_x[-hidden_size:]
output_weights = np.dot(pinv2(hidden_nodes(rfe_X_train)), y_train)
print(output_weights.shape)

(20,)
<ipython-input-117-13c3d2f2b0f1>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [118]:
def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

prediction = predict(rfe_X_test)
mean_squared_error(y_test,prediction)

<ipython-input-117-13c3d2f2b0f1>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


44.19176599137174